In [1]:
import pickle
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_recall_fscore_support
from sklearn.model_selection import KFold, cross_val_score
from sklearn.multiclass import OneVsRestClassifier  # support from multiclass
import time
from sklearn.svm import SVC

In [29]:
def open_pickle_file(path, pickle_file):
    file_loc = os.path.join(path, pickle_file)
    pickle_to_file = pickle.load(open(file_loc, "rb"),encoding='latin1')
    return pickle_to_file

In [30]:
data_path = '/media/ak/DataOnly/FinDataReal/'
symbols_list = list(os.listdir(data_path))
sample_symbol_path = os.path.join(data_path, symbols_list[1],'MODEL_BASED')
sample_symbol_dates =os.listdir(sample_symbol_path)
sample_features_date_file = os.path.join(sample_symbol_path, sample_symbol_dates[1])
sample_features_file =os.listdir(sample_features_date_file)[3]

open_pickle_file(sample_features_date_file, sample_features_file)

(      fischer_score_dlambda  fischer_score_dsigma  fischer_score_dweight
 0              2.036546e-08              0.000000               1.558872
 1              6.486815e+00              0.000000               2.609999
 2              2.482551e+01          -1156.951571              -3.348977
 3              2.661036e+01          -1156.951572              -1.817591
 4              3.409972e+01          -1156.951572              -0.620996
 5              9.173061e+01          -2312.675758              -6.617596
 6              1.007361e+02          -3470.699207             -12.572610
 7              1.173194e+02          -3470.699206             -11.268694
 8              1.745982e+02          -4625.803792             -17.304320
 9              1.766029e+02          -4626.207569             -15.768274
 10             2.301232e+02          -5781.348307             -21.801107
 11             2.794953e+02          -5781.811014             -20.591524
 12             2.814205e+02          

In [41]:
# raw_data_path ='/media/ak/DataOnly/Data/'
# raw_data_files =os.listdir(os.path.join(raw_data_path,os.listdir(raw_data_path)[0]))
# pd.read_csv(os.path.join(raw_data_path, os.listdir(raw_data_path)[0],raw_data_files[0]))
#os.listdir('/media/ak/DataOnly/Data/features_models/models/')
open_pickle_file('/media/ak/DataOnly/Data/features_models/models/', 'clfs_model_hash.pickle')

{'all_symbol_locations': defaultdict(dict,
             {'STAN.L': {0: [('/media/ak/WorkDrive/Data/features_models/labels/STAN.L/NON_DIRECTIONAL/20180315.csv',
                 '/media/ak/WorkDrive/Data/features_models/features/STAN.L/MODEL_BASED/20180314/STAN.L_3_states_features_date:_20180315_now:_20190109_.pickle'),
                ('/media/ak/WorkDrive/Data/features_models/labels/STAN.L/NON_DIRECTIONAL/20180315.csv',
                 '/media/ak/WorkDrive/Data/features_models/features/STAN.L/MODEL_BASED/20180314/STAN.L_3_states_features_date:_20180315_now:_20190109_.pickle'),
                ('/media/ak/WorkDrive/Data/features_models/labels/STAN.L/NON_DIRECTIONAL/20180316.csv',
                 '/media/ak/WorkDrive/Data/features_models/features/STAN.L/MODEL_BASED/20180314/STAN.L_3_states_features_date:_20180316_now:_20190109_.pickle'),
                ('/media/ak/WorkDrive/Data/features_models/labels/STAN.L/NON_DIRECTIONAL/20180316.csv',
                 '/media/ak/WorkDrive/Data/fe

In [78]:
data_path = '/media/ak/DataOnly/FinDataReal/'
symbolIdx=1

 #,'MODEL_BASED')

#symbol=os.listdir(labels_path)[symbolIdx]



In [227]:

class DataLoader(object):
    """
    class that loads and cleans up data 
    constructor params: path_main
    todo: needs some refactoring so i dont produce the same function 2x and i can read from predecessors
    """
    
    def __init__(self, path_main, ticker):
        self.main_path = path_main
        self.ticker = ticker
        
        # collection of per symbol non directional labels
        self.symbol_labels_path = os.path.join(self.main_path, 'Labels', self.ticker, 'NON_DIRECTIONAL')
        self.symbol_features_path = os.path.join(self.main_path, self.ticker, 'MODEL_BASED')
        # list of all the model -oos hmm feature dates - each folder is a collection of oos feature dates
        self.hmm_dates_list = os.listdir(self.symbol_features_path)  # each folder are the OOS features from each HMM
        self.compute_date = os.listdir(os.path.join( \
            self.symbol_features_path, \
            os.listdir(self.symbol_features_path)[1]))[1].split("_")[7]

    def symbol_features_date_load(self, hmm_model_date, features_date):
        """
        hmm_model_date: the hmm model that was used to produce the features, it is hashed by the date
        features_date: the features date that we want to retrieve the features for
        return: set of features for 3 state hmm for hmm_model_date, features_date combination
        """
        # need to make this a lot more flexible with number of states
        if hmm_model_date < features_date: #condition needed to make sure we dont pick-insample data
            featuresfilename = "_".join(
                (self.ticker, '3', 'states', 'features', 'date:', str(features_date), 'now:', self.compute_date, '.pickle'))
            file_loc = os.path.join(self.symbol_features_path, str(hmm_model_date), featuresfilename)
            
            with open(file_loc, 'rb') as handle:
                
                ticker_features_date = pickle.load(handle, encoding='latin1')
                
        if hmm_model_date > features_date:
            raise ValueError('Loading Feature Date which is in-sample. Change your Model Date')
            
        return ticker_features_date

    def symbol_labels_date(self, date):
        """
        takes a specific date that labels exist and returns the exact labels from that dataframe
        parameter: date that labels exist
        
        """        
        file_loc = os.path.join(self.symbol_labels_path, str(date) + '.csv')
        symbol_date_df = pd.read_csv(file_loc, index_col=0)
        return symbol_date_df.filter(like='label', axis=1)
    
    def symbol_date_processed_data_load(self, date):
        """
        takes a specific date that labels exist and returns the processed data frame of raw data
        parameter: date that labels exist
        
        """        
        return pd.read_csv(os.path.join(self.symbol_labels_path, str(date) + '.csv'))
        

    @staticmethod
    def open_pickle_file(path, pickle_file):
        file_loc = os.path.join(path, pickle_file)
        pickle_to_file = pickle.load(open(file_loc, "rb"),encoding='latin1')
        return pickle_to_file

    @staticmethod
    def get_date_from_file(file_, numb_):
        return os.path.splitext(file_[numb_])[0]



In [228]:
testSymbol = 'SMIN.L'
TestingDataLoader = DataLoader(data_path, str(testSymbol))

In [229]:
TestDFLoaded=TestingDataLoader.symbol_date_processed_data_load('20170704')
TestDFLoaded.head(4)


,Unnamed: 0,TradedTime,TradedPrice,ReturnTradedPrice,Volume,Duration,label_PrMov__window_5__thres_arbitrary__0.1
0,0,08:00:08.013564,16.06,0.000000,400.0,0.000000,0.0
1,1,08:00:08.013589,16.06,0.000000,1100.0,0.000025,1.0
2,2,08:00:29.772373,16.06,0.000000,228.0,21.758784,1.0
3,3,08:01:38.379568,16.07,0.000622,78.0,68.607195,0.0


In [203]:
idx= 1
path =os.path.join(symbol_features_path,os.listdir(symbol_features_path)[idx])
features_list = os.listdir(os.path.join(symbol_features_path,os.listdir(symbol_features_path)[idx]))

In [215]:
symbol_features_dates_list = os.listdir(os.path.join(data_path,symbol,'MODEL_BASED'))
symbol_features_path = os.path.join(data_path,symbol,'MODEL_BASED')

In [216]:
symbol_features_date_sub_path =os.path.join(symbol_features_path, symbol_features_dates_list[0])
print(symbol_features_date_sub_path)

/media/ak/DataOnly/FinDataReal/SMIN.L/MODEL_BASED/20180306


In [217]:
os.listdir(symbol_features_date_sub_path)[0:10]

['SMIN.L_3_states_features_date:_20180321_now:_20181223_.pickle',
 'SMIN.L_3_states_features_date:_20180411_now:_20190109_.pickle',
 'SMIN.L_3_states_features_date:_20180326_now:_20181223_.pickle',
 'SMIN.L_3_states_features_date:_20180329_now:_20181223_.pickle',
 'SMIN.L_3_states_features_date:_20180403_now:_20181223_.pickle',
 'SMIN.L_3_states_features_date:_20180314_now:_20181223_.pickle',
 'SMIN.L_3_states_features_date:_20180410_now:_20190109_.pickle',
 'SMIN.L_3_states_features_date:_20180320_now:_20181223_.pickle',
 'SMIN.L_3_states_features_date:_20180322_now:_20181223_.pickle',
 'SMIN.L_3_states_features_date:_20180307_now:_20181223_.pickle']

In [236]:
TestingDataLoader.symbol_features_date_load('20180306', '20180320')

,fischer_score_dlambda,fischer_score_dsigma,fischer_score_dweight
0,1.864515e-16,0.000000,1.689195
1,-1.013904e+02,3497.794163,-0.316171
2,-8.095660e+01,3497.794163,1.678466
3,-1.374784e+02,6995.588326,-0.326901
4,-1.170432e+02,6995.588326,1.667761
5,-1.014543e+02,7364.983203,-0.443288
6,-8.112287e+01,7390.646112,1.549328
7,-6.300748e+01,6435.323743,-0.456039
8,-4.388065e+01,5480.952181,-2.461406
9,-3.130519e+01,8990.139635,-4.466772


In [ ]:
def symbol_features_date_load(self, hmm_model_date, features_date):
        # need to make this a lot more flexible with number of states
        if hmm_model_date < features_date: #condition needed to make sure we dont pick-insample data
            featuresfilename = "_".join(
                (self.ticker, '3', 'states', 'features', 'date:', str(features_date), 'now:', self.compute_date, '.pickle'))
            file_loc = os.path.join(self.symbol_features_path, str(hmm_model_date), featuresfilename)
            
            with open(file_loc, 'rb') as handle:
                
                ticker_features = pickle.load(handle,encoding='latin1')
        else:
            
            print('Loading Feature Date which is in-sample. Change your Model Date')
        
        return file_loc
